<a href="https://colab.research.google.com/github/manuelarangom/MBrownianG/blob/master/random_walk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import io
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# PARÁMETROS

ticker = input( 'Ticker: ' )
days = int( input( 'Days: ' ) )
scenarios = int( input( 'Scenarios: ' ) )

In [0]:
# CONSULTAR HISTÓRICO DE PRECIOS DE ALPHAVANTAGE

def get_prices(ticker):

  data = ""
  url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={mnemo}&outputsize=full&apikey=YOURALPHAVANTAGEAPIHERE'.format(mnemo=ticker)
  response = requests.get(url,data=data)
  datos = response.json()
  stock_history = pd.DataFrame([ (k,float(v['1. open']),float(v['2. high']),float(v['3. low']),float(v['4. close']),int(v['5. volume'])) for k,v in datos['Time Series (Daily)'].items()], columns = ['Date','Open','High','Low','Close','Volume'])
  return stock_history

In [0]:
# CALCULAR RENDIMIENTOS LOGARÍTMICOS

def get_log_returns(full_data):

  log_returns = []
  [ log_returns.append(np.log( full_data['Close'][p] / full_data['Close'][ p+1 ] )) for p in range(0, len( full_data )-1 )]
  
  return log_returns

In [0]:
# CALCULAR REDIMIENTO PROMEDIO Y DESVIACIÓN ESTÁNDAR ANUALES SOBRE LOS RENDIMIENTOS

def stock_features(log_returns,tf):

  stock_features = {

    'avg_return':(sum(log_returns) / len(log_returns)) * int(tf),
    'std_return':np.std(log_returns) * np.sqrt(int(tf))

  }

  return stock_features

In [0]:
historic_prices = get_prices( ticker )

In [0]:
logarithmic_returns = get_log_returns(historic_prices)

In [0]:
stock_features = stock_features(logarithmic_returns,days)

Random Walk

In [0]:
import random
import scipy.stats as st
import math

random_prices = np.zeros((days,scenarios))

for esc in range(0,scenarios):

  random_prices[0][esc] = historic_prices['Close'][0]

  for dia in range(1,days):

    random_prices[dia][esc] = round( random_prices[dia-1][esc]* math.exp( ( stock_features['avg_return']-stock_features['std_return']**2 / 2 ) * (1/days) + stock_features['std_return']*(1/(days)**(1/2) ) * st.norm.ppf( random.uniform(0,1) ) ) , 2 )

In [0]:
random_prices = pd.DataFrame(random_prices)
random_prices

In [0]:
pr = random_prices.loc[200:200]
print(pr.sum(axis=1)/len(pr.keys()))

In [0]:
x = list(range(0,252))

figure = plt.figure(num=None, figsize=(20, 6), dpi=80, facecolor='w', edgecolor='g')
axes = figure.add_axes([2,2,2,2])

for k in random_prices.keys():
  axes.plot(x, random_prices[k])


axes.set_xlabel('Price')
axes.set_ylabel('Time')
axes.set_title('Possible Scenarios')